**This is a demo of how to reduce DRRP data using measurements of the Q stokes parameter in addition to the regular intensity, I.**

This analysis requires a Wollaston prism to split the beam into vertical (90 degrees) and horizontal (0 degrees) polarized rays onto the detector.

All of the tools you need are in the file [`drrp_functions`](https://github.com/UCSB-Exoplanet-Polarimetry-Lab/derp_control/blob/Q_reduction/Q_reduction/drrp_functions.py). The most convenient of these are wrapped into the function `multi_analysis()` which performs both I and Q analysis of DRRP data. For this demo, I will work with real data taken from the JHK half-wave plate. If you have access to the computer where measurements were stored, you can calculate I and Q from the raw data using extract_intensities. Otherwise, I have stored this information in .json files that can be used locally. 

In [19]:
import numpy as np
import json
from drrp_functions import extract_intensities, multi_analysis

If you have access to the raw images, run the following code to extract the data from calibration and sample measurements. Update the file paths as needed. You can also uncomment the last portions to create new .json files yourself, although this isn't necessary because I have provided them already. 

*Skip this step if you want to work with the .json files already provided.*

In [ ]:
wavelengths = [1100, 1200, 1300, 1400, 1500, 1600, 1750, 1850, 1950] # List of wavelengths to process

# Constants that don't change across wavelengths
reduced_filename = '' # can include the start of the filename, like 'Reduced_', or not
lcenter = [198, 194] # pixel coordinates [y, x] for the center of the left beam
rcenter = [198, 391]
maxradius = 65 # radius of the beam in pixels

# Function to convert NumPy arrays to lists
def convert_to_serializable(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

# CALIBRATION DATA
cal_results = {} # Initialize a dictionary to store the results

for wl in wavelengths:
    reduced_folder = f"D:\\desktop_drrp_data\\test_12_17_2024\\calibration\\calibration_raw\\Cal_{wl}_Reduced"
    extracted_data = extract_intensities(reduced_filename, reduced_folder, lcenter, rcenter, maxradius)
    
    # Store the data in the dictionary with keys that incorporate the wavelength
    cal_results[f'Cal_Il_{wl}'] = extracted_data[0]
    cal_results[f'Cal_Ir_{wl}'] = extracted_data[1]
    cal_results[f'Cal_theta{wl}'] = extracted_data[2]

# # UNCOMMENT TO CREATE A JSON FILE WITH THE CALIBRATION RESULTS
# cal_results_serializable = {key: convert_to_serializable(value) for key, value in cal_results.items()}
# # output_path = 'D:\\desktop_drrp_data\\test_12_17_2024\\extracted_data\\cal_results.json'
# output_path = './cal_results.json'
# with open(output_path, 'w') as jsonfile:
#     json.dump(cal_results_serializable, jsonfile)


# SAMPLE DATA
sample_results = {} # Initialize a dictionary to store the results

for wl in wavelengths:
    reduced_folder = f"D:\\desktop_drrp_data\\test_12_17_2024\\JHK_waveplate_measurement\\middle_position\\Cal_{wl}_Reduced"
    extracted_data = extract_intensities(reduced_filename, reduced_folder, lcenter, rcenter, maxradius)
    
    # Store the data in the dictionary with keys that incorporate the wavelength
    sample_results[f'Il_{wl}'] = extracted_data[0]
    sample_results[f'Ir_{wl}'] = extracted_data[1]
    sample_results[f'theta{wl}'] = extracted_data[2]

# # UNCOMMENT TO CREATE A JSON FILE WITH THE SAMPLE RESULTS
# sample_results_serializable = {key: convert_to_serializable(value) for key, value in sample_results.items()}
# output_path = './sample_results.json'
# with open(output_path, 'w') as jsonfile:
#     json.dump(sample_results_serializable, jsonfile)

Once you have the data organized, you're ready to start the analysis! 

Lets use the provided files and `multi_analysis()`. This function takes inputs for an array of wavelengths where data was taken, the calibration data, the sample data, and an option to choose data reduction method by specifying 'I' or 'q'. 

In [20]:
wavelengths = [1100, 1200, 1300, 1400, 1500, 1600, 1750, 1850, 1950]
cal_results = './cal_results.json'
sample_results = './sample_results.json'
results = multi_analysis(wavelengths, cal_results, sample_results, 'q')

Great! Now 'results' should be populated with the retardances, retardance errors, calibration matrices, sample matrices, and matrix RMS error for each wavelength. Results may differ slightly depending on whether 'I' or 'q' was chosen. 

If you want to access all of this data (like for making a plot) try: 

In [7]:
results = multi_analysis(wavelengths, cal_results, sample_results, 'q')
Q_retardances = []
Q_RMS_errors = []
Q_retardance_errors = []

for wavelength in wavelengths:
    retardance_value = results[wavelength][1]
    RMS_error = results[wavelength][3]
    
    Q_retardances.append(retardance_value)
    Q_RMS_errors.append(RMS_error)
    Q_retardance_errors.append(results[wavelength][4])

print(Q_retardances) # values in waves
print(Q_RMS_errors) # RMS error from the calibration Mueller matrix (unitless)
print(Q_retardance_errors) # values in waves

[0.46718401903046786, 0.48039806057167556, 0.4862502893881064, 0.4954119536691848, 0.4931415463942969, 0.4899002771286451, 0.49478835369717206, 0.49187835816436926, 0.4818261129387721]
[0.00952110565647759, 0.0033976581385555616, 0.0008057448668132094, 0.0013081668117417213, 0.001134459670875703, 0.0008620657633007036, 0.001011688890529457, 0.004072944513768305, 0.01939005984562204]
[0.007401570583291427, 0.004401689732083955, 0.001380898554079332, 0.007223301601456984, 0.004191191863404862, 0.0035507443234435625, 0.003581578735473481, 0.012708474643093815, 0.027084187823735808]


Let's say you want to get a good look at the data from a certain wavelength. That's easy too! Just adjust the wavelength array as needed:

In [ ]:
# example usage for 1300 nm, 'q' analysis
results = multi_analysis([1300], cal_results, sample_results, 'q')
matrix1300 = results[1300][0]
calmatrix1300 = results[1300][2]
RMS1300 = results[1300][3]
print(calmatrix1300, "Calibration matrix (Mueller matrix of air).")
print(matrix1300, "Mueller matrix of the sample.")
print("RMS error of the calibration Mueller matrix (unitless):", RMS1300)
print("Measured retardance in waves: ", results[1300][1])
print("Measured retardance error in waves: ", results[1300][4])

[[ 1.         -0.         -0.         -0.        ]
 [ 0.00000454  1.00062991  0.00063239  0.0012414 ]
 [-0.00116635  0.00216155  1.00046386 -0.00076601]
 [ 0.00006627 -0.00037372  0.00071461  0.99925136]] Calibration matrix (Mueller matrix of air).
[[ 1.         -0.         -0.         -0.        ]
 [ 0.00149678  0.97986183 -0.19616838 -0.00958264]
 [ 0.00115056 -0.19633003 -0.98391401 -0.08562799]
 [ 0.00010397  0.00695964  0.08485329 -0.99627053]] Mueller matrix of the sample.
RMS error of the calibration Mueller matrix (unitless): 0.0008057448668132094
Measured retardance in waves:  0.4862502893881064
Measured retardance error in waves:  0.001380898554079332


If we want to use 'I' for our data reduction, the results will look slightly different because we actually have two sets of results (one for each beam on the detector). 

In [18]:
# example usage for 1300 nm, 'I' analysis
results = multi_analysis([1300], cal_results, sample_results, 'I')
for wl, data in results.items():
        print(f"Wavelength: {wl}")
        print(f"{data[0]}, Mueller matrix of the sample (left).")
        print(f"{data[1]}, Mueller matrix of the sample (right).")
        print(f"Retardance of the sample (left): {data[2]}")
        print(f"Retardance of the sample (right): {data[3]}")
        print(f"Retardance error (left): {data[4]}")
        print(f"Retardance error (right): {data[5]}")
        print(f"{data[6]}, Mueller matrix of the calibration (left).")
        print(f"{data[7]}, Mueller matrix of the calibration (left).")
        print(f"RMS error of the left calibration matrix: {data[8]}")
        print(f"RMS error of the right calibration matrix: {data[9]}")
        print(f"Fit parameters (left) for a1, a2, w1, w2, r1, and r2. 1 for generator, 2 for analyzer: {data[10]}")
        print(f"Fit parameters (right) for a1, a2, w1, w2, r1, and r2. 1 for generator, 2 for analyzer: {data[11]}")

Wavelength: 1300
[[ 1.          0.00069107 -0.00030918 -0.00039326]
 [ 0.00267222  0.99638479  0.07308879  0.00154941]
 [ 0.00151482  0.07349541 -0.99964877 -0.0861174 ]
 [ 0.00010636 -0.00456301  0.08492651 -0.99524231]], Mueller matrix of the sample (left).
[[ 1.         -0.00069946  0.00029376  0.00039407]
 [ 0.00264202  0.99399262  0.11930378  0.00354349]
 [ 0.00157934  0.11971124 -0.99726335 -0.08623676]
 [ 0.00010658 -0.00653499  0.08497404 -0.99730138]], Mueller matrix of the sample (right).
Retardance of the sample (left): 0.4938517971605998
Retardance of the sample (right): 0.4883059460194393
Retardance error (left): 0.006959424410248614
Retardance error (right): 0.004522141556459223
[[ 1.          0.00160198  0.00164672  0.0000085 ]
 [-0.00156802  1.0036486   0.00272937  0.00134586]
 [-0.00139473  0.00242776  1.00247473 -0.00059057]
 [ 0.00006704 -0.00047075  0.00065798  1.00059199]], Mueller matrix of the calibration (left).
[[ 1.         -0.00155936 -0.00167874 -0.00000848]

This should be all the information you need. Fit parameters refers to the best fit values used in the calibration procedure. For more information on this process, see [`DRRP_Calibration_Demo.ipynb`](https://github.com/UCSB-Exoplanet-Polarimetry-Lab/derp_control/blob/Q_reduction/Calibration/DRRP_Calibration_Demo.ipynb).